In [1]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import matplotlib.pyplot as plt
from scipy.sparse import csr_matrix
from scipy.sparse.linalg import svds
from implicit.als import AlternatingLeastSquares

In [11]:
# STEP 1A: LOAD YAHOO & COAT DATASET


def load_yahoo_data():
    """
    Load Yahoo! R3 dataset from predefined train, validation, and test files.
    """

    train = pd.read_csv("./yahoo/trainset.csv")
    valid = pd.read_csv("./yahoo/validset.csv")
    test = pd.read_csv("./yahoo/testset.csv")
    train = train[['user_id', 'item_id', 'rating']]
    valid = valid[['user_id', 'item_id', 'rating']]
    test = test[['user_id', 'item_id', 'rating']]

    train.columns = valid.columns = test.columns = ['user_id', 'item_id', 'rating']
    return train, valid, test


def load_coat_data():
    """
    Load Coat dataset from predefined train, validation, and test files.
    """
    train = pd.read_csv("./coat/trainset.csv")
    valid = pd.read_csv("./coat/validset.csv")
    test = pd.read_csv("./coat/testset.csv")
    train.columns = valid.columns = test.columns = ['user_id', 'item_id', 'rating']
    return train, valid, test


In [3]:
# STEP 1B: SYNTHETIC DATASET GENERATION

def generate_synthetic_data(num_users=100, num_items=50, num_ratings=500):
    user_ids = np.random.randint(1, num_users + 1, num_ratings)
    item_ids = np.random.randint(1, num_items + 1, num_ratings)
    ratings = np.random.randint(1, 6, num_ratings)
    return pd.DataFrame({'user_id': user_ids, 'item_id': item_ids, 'rating': ratings})

In [4]:
# STEP 2: PROPENSITY SCORE COMPUTATION

def compute_propensities(df):
    item_popularity = df['item_id'].value_counts(normalize=True)
    rating_distribution = df['rating'].value_counts(normalize=True)
    df['popularity_propensity'] = df['item_id'].map(lambda x: item_popularity.get(x, 0.01))
    df['positivity_propensity'] = df['rating'].map(lambda x: rating_distribution.get(x, 0.01))
    df['multifactorial_propensity'] = df['popularity_propensity'] * df['positivity_propensity']
    df['ips_weight'] = 1 / (df['multifactorial_propensity'] + 1e-6)
    df['weighted_rating'] = df['rating'] * df['ips_weight']
    return df

In [5]:
# STEP 3A: MATRIX FACTORIZATION WITH SVD & ALS

def baseline_recommender(train, test):
    item_means = train.groupby('item_id')['rating'].mean()
    return test['item_id'].map(lambda x: item_means.get(x, 3))

def svd_recommender(train, test, k=20, debias=False):
    column = 'weighted_rating' if debias else 'rating'
    user_means = train.groupby('user_id')[column].mean()
    train['rating_normalized'] = train.apply(lambda x: x[column] - user_means.get(x['user_id'], 3), axis=1)
    user_item_matrix = train.pivot_table(index='user_id', columns='item_id', values=column, aggfunc='mean', fill_value=0)
    sparse_matrix = csr_matrix(user_item_matrix)
    U, sigma, Vt = svds(sparse_matrix, k=k)
    sigma = np.diag(sigma)
    predicted_ratings = np.dot(np.dot(U, sigma), Vt)
    
    predictions = []
    num_users, num_items = predicted_ratings.shape
    
    for _, row in test.iterrows():
        user_idx = min(max(int(row['user_id']) - 1, 0), num_users - 1)
        item_idx = min(max(int(row['item_id']) - 1, 0), num_items - 1)
        predictions.append(predicted_ratings[user_idx, item_idx])
    
    return np.clip(predictions, 1, 5)

def als_recommender(train, test, factors=20, debias=False):
    column = 'weighted_rating' if debias else 'rating'
    user_item_matrix = csr_matrix(train.pivot_table(index='user_id', columns='item_id', values=column, aggfunc='mean', fill_value=0))
    model = AlternatingLeastSquares(factors=factors, iterations=20)
    model.fit(user_item_matrix)
    
    predictions = []
    num_users, num_items = user_item_matrix.shape
    
    for _, row in test.iterrows():
        user_idx = min(max(int(row['user_id']) - 1, 0), num_users - 1)
        item_idx = min(max(int(row['item_id']) - 1, 0), num_items - 1)
        predictions.append(model.user_factors[user_idx] @ model.item_factors[item_idx].T)
    
    return np.clip(predictions, 1, 5)

In [6]:
# Neural Collaborative Filtering (NCF)
class NCF(nn.Module):
    def __init__(self, num_users, num_items, embedding_dim=20):
        super(NCF, self).__init__()
        self.user_embedding = nn.Embedding(num_users, embedding_dim)
        self.item_embedding = nn.Embedding(num_items, embedding_dim)
        self.fc = nn.Linear(embedding_dim * 2, 1)

    def forward(self, user_ids, item_ids):
        user_emb = self.user_embedding(user_ids)
        item_emb = self.item_embedding(item_ids)
        x = torch.cat([user_emb, item_emb], dim=-1)
        return self.fc(x).squeeze()


def train_ncf(train, valid, num_epochs=10, embedding_dim=20):
    num_users = int(train['user_id'].max()) + 1
    num_items = int(train['item_id'].max()) + 1
    model = NCF(num_users, num_items, embedding_dim)
    criterion = nn.MSELoss()
    optimizer = optim.Adam(model.parameters(), lr=0.001)
    
    train_users = torch.tensor(train['user_id'].values, dtype=torch.long)
    train_items = torch.tensor(train['item_id'].values, dtype=torch.long)
    train_ratings = torch.tensor(train['rating'].values, dtype=torch.float)
    
    for epoch in range(num_epochs):
        model.train()
        optimizer.zero_grad()
        predictions = model(train_users, train_items)
        loss = criterion(predictions, train_ratings)
        loss.backward()
        optimizer.step()
        
    return model

In [7]:
# STEP 4: EVALUATION

def evaluate(test, predictions):
    mse = mean_squared_error(test['rating'], predictions)
    mae = mean_absolute_error(test['rating'], predictions)
    rmse = np.sqrt(mse)
    r2 = r2_score(test['rating'], predictions)
    return mse, mae, rmse, r2


In [8]:
# ----------------------------
# STEP 6: EXPERIMENTATION
# ----------------------------

def run_experiment(dataset='yahoo', use_synthetic=False):
    if use_synthetic:
        train = generate_synthetic_data()
        valid = generate_synthetic_data()
        test = generate_synthetic_data()
    elif dataset == 'yahoo':
        train, valid, test = load_yahoo_data()
    elif dataset == 'coat':
        train, valid, test = load_coat_data()
    
    train = compute_propensities(train)
    valid = compute_propensities(valid)
    test = compute_propensities(test)
    
    best_k = 20
    
    svd_predictions = svd_recommender(train, test, k=best_k, debias=False)
    svd_debias_predictions = svd_recommender(train, test, k=best_k, debias=True)
    als_predictions = als_recommender(train, test, factors=best_k, debias=False)
    als_debias_predictions = als_recommender(train, test, factors=best_k, debias=True)
    
    print("Without Bias Correction:")
    print(evaluate(test, svd_predictions))
    print(evaluate(test, als_predictions))
    print("With Bias Correction:")
    print(evaluate(test, svd_debias_predictions))
    print(evaluate(test, als_debias_predictions))

if __name__ == "__main__":
    run_experiment(dataset='yahoo', use_synthetic=False)
    run_experiment(dataset='coat', use_synthetic=False)
    run_experiment(dataset='', use_synthetic=True)

/Users/sara.thibierge/Library/Python/3.9/lib/python/site-packages/implicit/cpu/als.py:95: RuntimeWarning: OpenBLAS is configured to use 8 threads. It is highly recommended to disable its internal threadpool by setting the environment variable 'OPENBLAS_NUM_THREADS=1' or by calling 'threadpoolctl.threadpool_limits(1, "blas")'. Having OpenBLAS use a threadpool can lead to severe performance issues here.
  check_blas_config()


  0%|          | 0/20 [00:00<?, ?it/s]

  0%|          | 0/20 [00:00<?, ?it/s]

Without Bias Correction:
(1.7946417055866035, 0.8217243790126296, np.float64(1.339642379736698), -0.6062386757104727)
(1.787207841873169, 0.8185670375823975, np.float64(1.3368649303026723), -0.5995852947235107)
With Bias Correction:
(9.016225495913346, 2.63211348382651, np.float64(3.002703031588929), -7.06969438823397)
(1.7782549858093262, 0.8188245296478271, np.float64(1.333512274337708), -0.5915722846984863)


  0%|          | 0/20 [00:00<?, ?it/s]

  0%|          | 0/20 [00:00<?, ?it/s]

Without Bias Correction:
(3.0494946971650925, 1.23084968994718, np.float64(1.7462802458841171), -0.9650009027960611)
(3.080980028116043, 1.2365731047386797, np.float64(1.7552720666939479), -0.9852890849007772)
With Bias Correction:
(7.355411823309893, 2.308710217526781, np.float64(2.7120862492387467), -3.739601904104669)
(3.064318243777848, 1.234049545171169, np.float64(1.7505194211370085), -0.9745527418282072)


  0%|          | 0/20 [00:00<?, ?it/s]

  0%|          | 0/20 [00:00<?, ?it/s]

Without Bias Correction:
(5.780225450070027, 1.9822708161656433, np.float64(2.404209943010391), -2.1043167923400627)
(5.869756698608398, 2.002556085586548, np.float64(2.422758076781171), -2.152400255203247)
With Bias Correction:
(6.2050329511248075, 2.086006155546017, np.float64(2.490990355486108), -2.332463093972709)
(5.869858741760254, 2.0019917488098145, np.float64(2.422779135984181), -2.1524550914764404)


In [10]:
def evaluate_ncf(model, test):
    model.eval()
    test_users = torch.tensor(test['user_id'].values, dtype=torch.long)
    test_items = torch.tensor(test['item_id'].values, dtype=torch.long)
    with torch.no_grad():
        predictions = model(test_users, test_items).numpy()
    return predictions

def run_experiment(dataset='yahoo', use_synthetic=False):
    if use_synthetic:
        train = generate_synthetic_data()
        valid = generate_synthetic_data()
        test = generate_synthetic_data()
    elif dataset == 'yahoo':
        train, valid, test = load_yahoo_data()
    elif dataset == 'coat':
        train, valid, test = load_coat_data()
    
    train = compute_propensities(train)
    valid = compute_propensities(valid)
    test = compute_propensities(test)
    
    best_k = 20
    
    ncf_model = train_ncf(train, valid)
    ncf_predictions = evaluate_ncf(ncf_model, test)
    
    print("Neural Collaborative Filtering (NCF):")
    print(evaluate(test, ncf_predictions))
    
if __name__ == "__main__":
    run_experiment(dataset='yahoo', use_synthetic=False)


Neural Collaborative Filtering (NCF):
(5.050364017486572, 1.9231197834014893, np.float64(2.2473014967926694), -3.520172595977783)
